**Import required libraries and scripts**

In [1]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.ranking_functions import *
from scripts.performance_calculation import *
import numpy as np
import os

software = '/home/tony/CADD22/software'
protein_file = '/home/tony/CADD22/wocondock_main/2o1x_A_apo_protoss.pdb'
ref_file = '/home/tony/CADD22/wocondock_main/2o1x_A_lig_protoss.sdf'
docking_library = '/home/tony/CADD22/wocondock_main/500_of_FCHGroup_LeadLike.sdf'
docking_programs = ['SMINA','GNINA','PLANTS']
id_column = 'ID'
n_poses = 10
exhaustiveness = 4

#Initialise variables and create a temporary folder
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
create_temp_folder(w_dir+'/temp')

[23:48:38] Initializing Normalizer


The working directory has been set to: /home/tony/CADD22/wocondock_main
The folder: /home/tony/CADD22/wocondock_main/temp already exists


In [ ]:
cleaned_pkasolver_df = prepare_library(docking_library, id_column, software, 'pkasolver')

In [ ]:
all_poses = docking(protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses)

In [ ]:
docking_splitted(w_dir, protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses)

In [5]:
cluster('espsim', w_dir, protein_file)
cluster('spyRMSD', w_dir, protein_file)
cluster('USRCAT', w_dir, protein_file)
cluster('RMSD', w_dir, protein_file)
cluster('3DScore', w_dir, protein_file)

Loading all poses SDF file...
Finished loading all poses SDF file...
Clustering folder already exists
*Calculating espsim metrics and clustering*


100%|██████████| 10/10 [00:13<00:00,  1.38s/it]


Loading all poses SDF file...
Finished loading all poses SDF file...
Clustering folder already exists
*Calculating spyRMSD metrics and clustering*


 10%|█         | 1/10 [00:02<00:22,  2.53s/it]

In [4]:
cluster_numpy('espsim', w_dir, protein_file)
cluster_numpy('spyRMSD', w_dir, protein_file)
cluster_numpy('USRCAT', w_dir, protein_file)
cluster_numpy('RMSD', w_dir, protein_file)
cluster_numpy('3DScore', w_dir, protein_file)

Loading all poses SDF file...
Finished loading all poses SDF file...
Clustering folder already exists
*Calculating espsim metrics and clustering*


100%|██████████| 10/10 [00:03<00:00,  2.91it/s]


Loading all poses SDF file...
Finished loading all poses SDF file...
Clustering folder already exists
*Calculating spyRMSD metrics and clustering*


100%|██████████| 10/10 [00:19<00:00,  1.94s/it]


Loading all poses SDF file...
Finished loading all poses SDF file...
Clustering folder already exists
*Calculating USRCAT metrics and clustering*


100%|██████████| 10/10 [02:02<00:00, 12.26s/it]


Loading all poses SDF file...
Finished loading all poses SDF file...
Clustering folder already exists
*Calculating RMSD metrics and clustering*


100%|██████████| 10/10 [00:04<00:00,  2.34it/s]


Loading all poses SDF file...
Finished loading all poses SDF file...
Clustering folder already exists
*Calculating 3DScore metrics and clustering*


100%|██████████| 10/10 [00:18<00:00,  1.84s/it]


In [2]:
cluster_numpy_futures('espsim', w_dir, protein_file)
cluster_numpy_futures('spyRMSD', w_dir, protein_file)
cluster_numpy_futures('USRCAT', w_dir, protein_file)
cluster_numpy_futures('RMSD', w_dir, protein_file)
cluster_numpy_futures('3DScore', w_dir, protein_file)

Loading all poses SDF file...
Finished loading all poses SDF file...
Clustering folder already exists
*Calculating espsim metrics and clustering*


10it [00:02,  3.74it/s]


Loading all poses SDF file...
Finished loading all poses SDF file...
Clustering folder already exists
*Calculating spyRMSD metrics and clustering*


10it [00:06,  1.63it/s]


Loading all poses SDF file...
Finished loading all poses SDF file...
Clustering folder already exists
*Calculating USRCAT metrics and clustering*


10it [00:54,  5.46s/it]


Loading all poses SDF file...
Finished loading all poses SDF file...
Clustering folder already exists
*Calculating RMSD metrics and clustering*


10it [00:01,  5.80it/s]


Loading all poses SDF file...
Finished loading all poses SDF file...
Clustering folder already exists
*Calculating 3DScore metrics and clustering*


10it [00:08,  1.12it/s]


**Rescoring**

The file containing all the cluster centers is then rescored using all scoring functions available (GNINA, Vina, AutoDock4, PLP, CHEMPLP, RF-Score-VS). The rescored output is return as a dataframe.

In [ ]:
RMSD_rescored = rescore_all(w_dir, protein_file, ref_file, software, w_dir+'/temp/clustering/RMSD_clustered.sdf')
espsim_rescored = rescore_all(w_dir, protein_file, ref_file, software, w_dir+'/temp/clustering/espsim_clustered.sdf')
spyRMSD_rescored = rescore_all(w_dir, protein_file, ref_file, software, w_dir+'/temp/clustering/spyRMSD_clustered.sdf')
USRCAT_rescored = rescore_all(w_dir, protein_file, ref_file, software, w_dir+'/temp/clustering/USRCAT_clustered.sdf')
DScore_rescored = rescore_all(w_dir, protein_file, ref_file, software, w_dir+'/temp/clustering/3DScore_clustered.sdf')
bestpose_rescored = rescore_all(w_dir, protein_file, ref_file, software, w_dir+'/temp/clustering/bestpose_clustered.sdf')

**Final ranking methods**

This code calculates the final ranking of compounds using various methods.
*Method 1* : Calculates ECR value for each cluster center, then outputs the top ranked center.
*Method 2* : Calculates ECR value for each cluster center, then outputs the average ECR value for each compound.
*Method 3* : Calculates the average rank of each compound, then ouputs the corresponding ECR value for each compound.
*Method 6* : Calculates Z-score for each cluster center, then ouputs the top ranked center.
*Method 7* : Calculates Z-score for each cluster center, then ouputs the average Z-score for each compound.

All methods are then combined into a single dataframe for comparison purposes.

In [ ]:
apply_ranking_methods_simplified(w_dir)

In [ ]:
test_df = pd.read_csv('/home/tony/CADD22/wocondock_refactored_chatgpt/temp/ranking/ranking_results.csv')
def show_correlation(dataframe):
    matrix = dataframe.corr().round(2)
    mask = np.triu(np.ones_like(matrix, dtype=bool))
    sns.heatmap(matrix, mask = mask, annot=False, vmax=1, vmin=-1, center=0, linewidths=.5, cmap='coolwarm')
    plt.show()

show_correlation(test_df)

In [ ]:
calculate_EFs(w_dir, docking_library)